# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Для начала загрузим все нужные нам библиотеки.

In [1]:
from tqdm import notebook
from tqdm.notebook import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import math

import re
from pymystem3 import Mystem
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

Откроим фаил и посмотрим общие данные.

In [2]:
try:
    data = pd.read_csv('C:/Users/Пользователь/OneDrive/Документы/Python/toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

display(data.head(3))
data.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Посмотрим на дисбаланс классов.

In [3]:
display(data['toxic'].value_counts())
data['toxic'].value_counts(normalize=True).map('{:.2%}'.format)

0    143106
1     16186
Name: toxic, dtype: int64

0    89.84%
1    10.16%
Name: toxic, dtype: object

Проверим на пропуски.

In [4]:
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Перейдем к подготовке признаков и разобъем датасет на выборки.

In [5]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
sentence = "The striped bats are hanging on their feet for best"

doc = nlp(sentence)

" ".join([token.lemma_ for token in doc])
from sklearn.ensemble import RandomForestClassifier

In [6]:
data['text'] = data['text'].values
data['text'] = data['text'].str.lower()

l = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize_text(phrase):
    phrase_l = l(phrase)
    lemma_phrase = " ".join([token.lemma_ for token in phrase_l])
    finished_text = re.sub(r'[^a-zA-Z]', ' ', lemma_phrase) 
    return " ".join(finished_text.split())

data['lemma_text'] = data['text'].progress_apply(lemmatize_text)

data.head(10)

  0%|          | 0/159292 [00:00<?, ?it/s]

,Unnamed: 0,text,toxic,lemma_text
0,0,explanation\nwhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,1,d'aww! he matches this background colour i'm s...,0,d aww he match this background colour I be see...
2,2,"hey man, i'm really not trying to edit war. it...",0,hey man I be really not try to edit war it be ...
3,3,"""\nmore\ni can't make any real suggestions on ...",0,more I can not make any real suggestion on imp...
4,4,"you, sir, are my hero. any chance you remember...",0,you sir be my hero any chance you remember wha...
5,5,"""\n\ncongratulations from me as well, use the ...",0,congratulation from I as well use the tool wel...
6,6,cocksucker before you piss around on my work,1,cocksucker before you piss around on my work
7,7,your vandalism to the matt shirvington article...,0,your vandalism to the matt shirvington article...
8,8,sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense be offensive to you...
9,9,alignment on this subject and which are contra...,0,alignment on this subject and which be contrar...


In [7]:
features = data['lemma_text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.2, 
                                                                              random_state=12345)

features_test, features_valid, target_test, target_valid = train_test_split(features_test, 
                                                                              target_test, 
                                                                              test_size=0.5, 
                                                                              random_state=12345)

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train.values)
features_valid = count_tf_idf.transform(features_valid.values)
features_test = count_tf_idf.transform(features_test.values)

print(f'features_train shape is {features_train.shape}')
print(f'features_valid shape is {features_valid.shape}')
print(f'features_test shape is {features_test.shape}')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


features_train shape is (127433, 133938)
features_valid shape is (15930, 133938)
features_test shape is (15929, 133938)


## Обучение

ПРедварительная работа сделана, перейдем к обучению моделей. Возьмем для сравнения несколько: LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, CatBoostClassifier.

In [8]:
model_lr = LogisticRegression(C=10, solver='lbfgs', max_iter=10000, class_weight=1).fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)
f1_score_lr = f1_score(target_valid, predictions_lr)
print(f'F1: {f1_score_lr}')

F1: 0.7909527073337903


In [9]:
model_dtc = DecisionTreeClassifier(max_depth=25).fit(features_train, target_train)
predictions = model_dtc.predict(features_valid)
f1_score_dct = f1_score(target_valid, predictions)
print(f'F1: {f1_score_dct}')

F1: 0.6736448598130841


In [10]:
model_rfc = RandomForestClassifier(n_estimators = 60, max_depth=50).fit(features_train, target_train)
predictions = model_rfc.predict(features_valid)
f1_score_rfc = f1_score(target_valid, predictions)
print(f'F1: {f1_score_rfc}')

F1: 0.05045045045045045


In [ ]:
model_cb = CatBoostClassifier(verbose=False, iterations=250).fit(features_train, target_train)
predictions = model_cb.predict(features_valid)
f1_score_cb_cv = cross_val_score(model_cb,
                                         features_train, 
                                         target_train, 
                                         cv=3, 
                                         scoring='f1').mean()
f1_score_cb = f1_score(target_valid, predictions)
print(f'F1: {f1_score_cb}')
print('F1 на кросс-валидации', f1_score_cb_cv)

In [ ]:
f1_scores = pd.DataFrame(data = [round(f1_score_lr, 2), round(f1_score_dct, 2), round(f1_score_rfc, 2), round(f1_score_cb, 2)],
                         index=['LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier', 'CatBoostClassifier'],
                         columns = ['F1'])
f1_scores = f1_scores.sort_values(by='F1', ascending=False)
display(f1_scores)

Очень неожиданно победителем оказалась модель LogisticRegression!! Такого раньше не было)
Не исключено, что при смене показателей гиперпараметров у модели CatBoostClassifier можно достич лучшего результата.

Поверим модель на тестовой выборке.

In [ ]:
predictions_lr = model_lr.predict(features_test)
f1_score_lr = f1_score(target_test, predictions_lr)
print(f'F1: {f1_score_lr}')

Все отлично, расхождения минимальны.

## Выводы

Задача выполнена. Мы обработали данный нам датасет, сформировали признаки и протестировали несколько моделей.
Показатели метрики F1 выше заданного, а значит хрупкая психика пользователей не достигших 18 лет и просто неравнодушных к родному языку находится под стражей нашего кода))